In [2]:
import json

In [3]:
word_idx = sc.textFile('hdfs://master:54310/word_idx'). \
    map(lambda line: json.loads(line)). \
    collectAsMap()

In [44]:
data = sc.textFile('hdfs://master:54310/clean'). \
    map(lambda line: json.loads(line))

In [39]:
from collections import Counter

def vectorize_text(words: list):
    counter = Counter(words)
    return [(word_idx[w], c) for w, c in counter.items() if w in word_idx]

In [50]:
def transform_data(data: dict):
    res = {}
    res['Features'] = vectorize_text(data['Text'])
    res['Labels'] = data['Hubs']
    res['Id'] = data['Id']
    return res

In [ ]:
data.map(lambda x: transform_data(x)). \
    filter(lambda x: x['Features']). \
    map(lambda x: json.dumps(x)). \
    repartition(4). \
    saveAsTextFile('hdfs://master:54310/experiment')